A

In [6]:
a, b, c, d = map(int, input().split())

if a < c:
    print('Takahashi')
elif a > c:
    print('Aoki')
else:
    if b <= d:
        print('Takahashi')
    else:
        print('Aoki')

Takahashi


B

In [10]:
n = int(input())
li = sorted(map(int, input().split()))

to_find = 0
for v in li:
    if v == to_find:
        to_find += 1

print(to_find)

3
0 2 3
1


C

In [13]:
n, k = map(int, input().split())
a = list(map(int, input().split()))
b = list(map(int, input().split()))

cands = {a[0], b[0]}
ip = True
for i in range(1, n):
    ncands = set()
    for fv in cands:
        if abs(a[i] - fv) <= k:
            ncands.add(a[i])
        if abs(b[i] - fv) <= k:
            ncands.add(b[i])
        
    if not ncands:
        ip = False
        break
    cands = ncands.copy()
    
if ip:
    print('Yes')
else:
    print('No')

4 1000000000
1 1 1000000000 1000000000
1 1000000000 1 1000000000
Yes


D

In [15]:
n, m = map(int, input().split())
a = list(map(int, input().split()))
c = list(map(int, input().split()))

b = [0] * (m + 1)
b[0] = c[0] // a[0]

for cc in range(1, n + m + 1):
    to_sub = 0
    to_divide = 0
    target = 0
    for aa in range(cc + 1):
        bb = cc - aa
        
        if aa > n or bb > m:
            continue
        if b[bb] == 0:
            target = bb
            to_divide = a[aa]
        else:
            to_sub += a[aa] * b[bb]
        
    if target == 0:
        continue
        
    c[cc] -= to_sub
    if to_divide:
        c[cc] //= to_divide
        
    b[target] = c[cc]
    
print(*b)

3 3
2 2 2 2
2 2 2 2 2 2 2
1 1 1 1


In [16]:
n, m = map(int, input().split())
a = list(map(int, input().split()))
b = list(map(int, input().split()))
c = list(map(int, input().split()))
d = list(map(int, input().split()))

li = [(a[i], b[i]) for i in range(n)]
boxes = [(c[i], d[i]) for i in range(m)]

li.sort(key=lambda x: (x[0], x[1]))
boxes.sort(key=lambda x: (x[0], x[1]))

2 3
2 4
3 2
8 1 5
2 10 5


In [5]:
import sys
input = lambda: sys.stdin.readline().rstrip()

# https://github.com/tatyam-prime/SortedSet/blob/main/SortedMultiset.py
import math
from bisect import bisect_left, bisect_right, insort
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedMultiset(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedMultiset from iterable. / O(N) if sorted / O(N log N)"
        a = list(a)
        if not all(a[i] <= a[i + 1] for i in range(len(a) - 1)):
            a = sorted(a)
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedMultiset" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def count(self, x: T) -> int:
        "Count the number of x."
        return self.index_right(x) - self.index(x)

    def add(self, x: T) -> None:
        "Add an element. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return
        a = self._find_bucket(x)
        insort(a, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True

    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError

    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return ans
    
n, m = map(int, input().split())
a = list(map(int, input().split()))
b = list(map(int, input().split()))
c = list(map(int, input().split()))
d = list(map(int, input().split()))

li = [(a[i], b[i]) for i in range(n)]
boxes = [(c[i], d[i]) for i in range(m)]

li.sort(key=lambda x: (x[0], x[1]))
boxes.sort(key=lambda x: (x[0], x[1]))
    
ms = SortedMultiset()
li_i = 0
for box_w, box_h in boxes:
    while li_i < n and li[li_i][0] <= box_w:
        ms.add(li[li_i][1])
        li_i += 1
    
    index = ms.index_right(box_h) - 1
    if index == -1:
        continue
    
    ms.discard(ms[index])
    
if ms or li_i < n:
    print('No')
else:
    print('Yes')

1 1
10
100
10
100
Yes


F

In [ ]:
import sys
# input = lambda: sys.stdin.readline().rstrip()

## PYRIVAL BOOTSTRAP
# https://github.com/cheran-senthil/PyRival/blob/master/pyrival/misc/bootstrap.py
# This decorator allows for recursion without actually doing recursion
## @bootstrap, yield when getting and returning value in recursive functions, end of functions
from types import GeneratorType

def bootstrap(f, stack=[]):
    def wrappedfunc(*args, **kwargs):
        if stack:
            return f(*args, **kwargs)
        else:
            to = f(*args, **kwargs)
            while True:
                if type(to) is GeneratorType:
                    stack.append(to)
                    to = next(to)
                else:
                    stack.pop()
                    if not stack:
                        break
                    to = stack[-1].send(to)
            return to
    return wrappedfunc

@bootstrap
def dfs(value):
    vt[value] = True
    temp.append(value)
    on_stack[value] = True
    
    for v in li[value]:
        if vt[v]:
            if is_cycle[v] or on_stack[v]:
                for vv in temp:
                    is_cycle[vv] = True
        else:
            yield dfs(v)
                
    temp.pop()
    on_stack[value] = False
    yield

n, m = map(int, input().split())
li = [[] for _ in range(n + 1)]
for _ in range(m):
    a, b = map(int, input().split())
    li[a].append(b)
    
is_cycle = [False] * (n + 1)
vt = [False] * (n + 1)
temp = []
on_stack = [False] * (n + 1)
for i in range(1, n + 1):
    if vt[i]:
        continue
    dfs(i)
    
r = 0
for i in range(1, n + 1):
    if is_cycle[i]:
        r += 1
        
print(r)